The following submodules are demonstrated in other tutorials:
- filtration (`2_3_clean_and_plot`)
- kde (used in `3_across_compounds`)
- spectra3d (`2_4_PS_spectra`)
- polychrom (TODO)

So we need to describe the following:
- select
- p3mhelpers
- vmd

# 1) Loading data

In [1]:
import xarray as xr
import shnitsel as st
import shnitsel.xarray

xr.set_options(
    display_expand_coords=False,
    display_expand_data_vars=False,
    display_expand_attrs=False,
)

In [2]:
from shnitsel.data.tree import tree_to_frames

db = st.io.read("tut_data/traj_I02.nc", kind="shnitsel", input_state_names=["S0", "S1", "S2"])
frames = tree_to_frames(db['I02'])
frames

<xarray.Dataset> Size: 6MB
Dimensions:                (state: 3, frame: 6452, atom: 14, direction: 3,
                            statecomb: 3, full_statecomb: 6, trajid_: 16)
Coordinates: (24)
Data variables: (9)
Attributes: (6)

# 2) Derive representation for selection

We will use PCA in this case, but could use any feature that associates each frame with two values to use as $x$ and $y$ coordinates in the selection plot.

In [3]:
pca = frames.atXYZ.st.pwdists().st.pca('atomcomb')
pca

INFO: Dimension atomcomb is being generated.


<xarray.DataArray (frame: 6452, PC: 2)> Size: 103kB
array([[ 0.2527689 ,  0.0462187 ],
       [ 0.25525838,  0.04271544],
       [ 0.25901722,  0.03414313],
       ...,
       [-0.64467528, -0.75732731],
       [-0.6684596 , -0.73634188],
       [-0.69243973, -0.71430794]], shape=(6452, 2))
Coordinates: (3)
Dimensions without coordinates: PC
Attributes: (5)

# 3) Frame-level selection
## 3.1) Select frames from PCA

When the cell below is run, an interactive Bokeh plot should appear.  
Please use it to select a part of the PCA you find interesting.

NB. Later on, we will use `py3Dmol` to visualize the points selected.  
On some systems, the underlying 3Dmol.js becomes very slow when

In [ ]:
selector = pca.st.FrameSelector()

Loading BokehJS ...

INFO: Starting Bokeh server version 3.8.1 (running on Tornado 6.5.2)
INFO: User authentication hooks NOT provided (default user enabled)


INFO: 200 GET /autoload.js?bokeh-autoload-element=fbd57c64-08fe-4269-8743-da121a173ab4&bokeh-absolute-url=http://localhost:45131&resources=none (::1) 54.93ms
INFO: 101 GET /ws?id=631edf33-e630-4468-b963-cf6797d382a0&origin=fde0f466-d350-4c7f-995b-8eef6d0b38d6&swVersion=4&extensionId=&platform=electron&vscode-resource-base-authority=vscode-resource.vscode-cdn.net&parentOrigin=vscode-file%3A%2F%2Fvscode-app&purpose=notebookRenderer (::1) 0.75ms
INFO: WebSocket connection opened
INFO: ServerConnection created


The indices of the selected points can be retrieved in several ways:

In [8]:
selector.selection
# selector.selected_frame_indices
# selector.df_selection

<xarray.DataArray (frame: 7, PC: 2)> Size: 112B
array([[-0.6565835 ,  0.79328341],
       [-0.65415134,  0.81079937],
       [-0.65194126,  0.82149068],
       [-0.64965961,  0.82451911],
       [-0.64702436,  0.81974782],
       [-0.64369177,  0.80757171],
       [-0.63920583,  0.78865899]])
Coordinates: (3)
Dimensions without coordinates: PC
Attributes: (5)

The indices can be used as follows, to retrieve the relevant subset of the full dataset:

In [9]:
subset = frames.isel(frame=selector.selected_frame_indices)
subset

<xarray.Dataset> Size: 10kB
Dimensions:                (state: 3, frame: 7, atom: 14, direction: 3,
                            statecomb: 3, full_statecomb: 6, trajid_: 16)
Coordinates: (24)
Data variables: (9)
Attributes: (6)

## 3.2) View geometries for selected frames

If the following gives the error `ValueError: frame must not be empty`, please ensure you have selected some points above.

In [11]:
subset.atXYZ.st.frames3Dgrid()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# 4) Trajectory-level selection
## 4.1) Select trajectories from PCA

There are two differences to the FrameSelector:
- when frames are selected, they show up as light green, but all other frames from their trajectory show up as dark green as well
- a live-updating list of selected trajectories is displayed under the plot

Please note that in Bokeh, additive selection is possible by holding Shift while clicking and dragging -- this may be particularly useful in selecting multiple trajectories.

In [12]:
selector = pca.st.TrajSelector()

Loading BokehJS ...

INFO: Starting Bokeh server version 3.8.1 (running on Tornado 6.5.2)
INFO: User authentication hooks NOT provided (default user enabled)


INFO: 200 GET /autoload.js?bokeh-autoload-element=d8f15268-5a43-4bfc-aa07-b861f3744ba8&bokeh-absolute-url=http://localhost:44865&resources=none (::1) 26.55ms
INFO:tornado.access:200 GET /autoload.js?bokeh-autoload-element=d8f15268-5a43-4bfc-aa07-b861f3744ba8&bokeh-absolute-url=http://localhost:44865&resources=none (::1) 26.55ms
INFO: 101 GET /ws?id=631edf33-e630-4468-b963-cf6797d382a0&origin=fde0f466-d350-4c7f-995b-8eef6d0b38d6&swVersion=4&extensionId=&platform=electron&vscode-resource-base-authority=vscode-resource.vscode-cdn.net&parentOrigin=vscode-file%3A%2F%2Fvscode-app&purpose=notebookRenderer (::1) 0.33ms
INFO:tornado.access:101 GET /ws?id=631edf33-e630-4468-b963-cf6797d382a0&origin=fde0f466-d350-4c7f-995b-8eef6d0b38d6&swVersion=4&extensionId=&platform=electron&vscode-resource-base-authority=vscode-resource.vscode-cdn.net&parentOrigin=vscode-file%3A%2F%2Fvscode-app&purpose=notebookRenderer (::1) 0.33ms
INFO: WebSocket connection opened
INFO: ServerConnection created


The selection info exposed by the selector is actually the same. In either case, it is easy to find which trajids have been selected:

In [13]:
import numpy as np
np.unique(selector.selection.trajid)

array([ 3, 12])

In [14]:
subset = frames.st.sel_trajs(selector.selection.trajid)
subset

<xarray.Dataset> Size: 795kB
Dimensions:                (state: 3, frame: 886, atom: 14, direction: 3,
                            statecomb: 3, full_statecomb: 6, trajid_: 2)
Coordinates: (24)
Data variables: (9)
Attributes: (6)

## 3.2) View geometries for selected trajectories

### 3.2.1) Using py3Dmol

In [ ]:
subset.atXYZ.st.trajs3Dgrid()

### 3.2.2) Using VMD
If you have the popular visualization program VMD installed, it is possible to open trajectories in it.
Initially, conformations from different trajectories will be overlapped.
Using the small extension window titled 'spread', it is possible to move them apart.
The ideal 'Separation' will depend on the desired zoom level and window size, but 1.5 is probably a reasonable value to start with.

In [ ]:
subset.atXYZ.st.traj_vmd()